In [1]:
!pip install tabula-py
import tabula

In [2]:
table = tabula.read_pdf("G1.pdf", pages=1)

In [3]:
for i in range(len(table)):
    table[i].columns = table[i].columns.str.replace(r"\r", " ")

In [4]:
table[0] = table[0].drop(columns=["Unnamed: 0"])

In [5]:
index = table[0].index
condition1 = table[0]['CHICOPEE BIG Y'] == "WEEKDAY"
condition2 = table[0]['CHICOPEE BIG Y'] == 'SATURDAY'
condition3 = table[0]['CHICOPEE BIG Y'] == 'SUNDAY'
day_indices = index[condition1][0], index[condition2][0], index[condition3][0]

table_weekday = table[0].iloc[day_indices[0] + 1: day_indices[1] - 1]
table_saturday = table[0].iloc[day_indices[1] + 1: day_indices[2] - 1]
table_sunday = table[0].iloc[day_indices[2] + 1: len(table[0])]
table_weekday.reset_index(drop=True, inplace=True)
table_saturday.reset_index(drop=True, inplace=True)
table_sunday.reset_index(drop=True, inplace=True)

In [11]:
def preprocess_table(table):
    for i in range(len(table.columns)):
        col = table.columns[i]
        table[col] = table[col].str.replace('[\|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|-]','')

In [12]:
preprocess_table(table_weekday)
preprocess_table(table_saturday)
preprocess_table(table_sunday)

C:\Users\quynh\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [14]:
# condition on mornings for weekdays. 
condition_weekday_morning = table_weekday['CHICOPEE BIG Y'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_weekday_morning = table_weekday.iloc[0: condition_weekday_morning]
table_weekday_morning.reset_index(drop=True, inplace=True)

In [15]:
# condition on evenings for weekdays
condition_weekday_evening = table_weekday['CHICOPEE BIG Y.1'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_weekday_evening = table_weekday.iloc[condition_weekday_evening:]
table_weekday_evening.reset_index(drop=True, inplace=True)

In [16]:
# condition on mornings for saturdays. 
condition_saturday_morning = table_saturday['CHICOPEE BIG Y'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_saturday_morning = table_weekday.iloc[0: condition_saturday_morning]
table_saturday_morning.reset_index(drop=True, inplace=True)

In [17]:
# condition on evenings for saturdays. 
condition_saturday_evening = table_saturday['CHICOPEE BIG Y.1'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_saturday_evening = table_saturday.iloc[condition_saturday_evening:]
table_saturday_evening.reset_index(drop=True, inplace=True)

In [18]:
# condition on mornings for sunday. 
condition_sunday_morning = table_sunday['CHICOPEE BIG Y'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_sunday_morning = table_weekday.iloc[0: condition_sunday_morning]
table_sunday_morning.reset_index(drop=True, inplace=True)

In [19]:
# condition on evenings for sunday. 
condition_sunday_evening = table_saturday['CHICOPEE BIG Y.1'].str.contains('[1][2]:[0-5][0-9]').idxmax()
table_sunday_evening = table_weekday.iloc[condition_sunday_evening:]
table_sunday_evening.reset_index(drop=True, inplace=True)

In [20]:
table_weekday_morning

,CHICOPEE BIG Y,CHICOPEE CENTER,UNION STATION BAY 2,"THE ""X""",RIVERBEND MEDICAL CENTER,ALLEN & COOLEY/ 5 TOWN PLAZA,CANON CIRCLE,ALLEN & COOLEY/ 5 TOWN PLAZA.1,"THE ""X"".1",UNION STATION BAY 6,CHICOPEE CENTER.1,CHICOPEE BIG Y.1
0,,,,,,,,,,,6:00,6:18
1,,,,,,,,5:35,5:45,6:10,6:25,6:43
2,,,,,,,,,,6:25,6:45,7:03
3,,,6:00,6:10,,,6:30,6:35,6:45,7:15,7:35,7:58
4,,,6:15,6:25,,,,,6:30,6:55,7:15,7:38
5,,,6:25,6:35,6:50,6:55,7:00,,7:10,7:40,8:00,8:23
6,,6:25,6:45,6:55,,,7:15,7:20,7:30,8:00,8:20,8:43
7,6:30,6:40,7:00,7:15,,,7:35,7:40,7:50,8:20,8:35,
8,6:45,7:00,7:20,7:35,7:45,7:55,8:00,,8:10,8:40,8:55,9:18
9,7:05,7:20,7:40,7:55,,,8:15,8:20,8:30,9:00,9:15,


In [21]:
table_weekday_morning.to_csv(r'weekday_morning.csv', index=False, header=True) 
table_weekday_evening.to_csv(r'weekday_evening.csv', index=False, header=True) 
table_saturday_morning.to_csv(r'saturday_morning.csv', index=False, header=True) 
table_saturday_evening.to_csv(r'saturday_evening.csv', index=False, header=True) 
table_sunday_morning.to_csv(r'sunday_morning.csv', index=False, header=True) 
table_sunday_evening.to_csv(r'sunday_evening.csv', index=False, header=True) 